# COSMIC DATA ANALYSIS

## Introduction

`COSMIC_csv_reader.py
COSMIC_sample_printer.py
pyfaidx_extraction.py
context_finder.py
threemer_count_jellyfish.sh
create_spectra.sh
create_uhc_dendrogram_heatmap.py`<br>
<br>
The objective of this project was to analyze the sequence contexts of point mutation data downloaded from the Catalogue of Somatic Mutations in Cancer (COSMIC) database (https://cancer.sanger.ac.uk/cosmic/download) and compare it to sequence context data from UVB exposure experiments conducted in the Essigmann lab.

## Descriptions and Methodology

### Filtering and outputting COSMIC point mutation data into separate .csv files

`COSMIC_csv_reader.py`<br>
Reads the downloaded .csv file from COSMIC, V85 and filters by certain parameters:

In [ ]:
if row[0] == "GENE_NAME": continue # Skips the header of the .csv
if row[PRIMARY_SITE] != "skin": continue
if row[PRIMARY_HISTOLOGY] != "malignant_melanoma": continue
if "del" in row[MUTATION_CDS] or "ins" in row[MUTATION_CDS]: continue # Ignore indels
if row[GRCH] == "null" or int(row[GRCH]) != 38: continue
if "null" in row[MUTATION_STRAND]: continue
if row[MUTATION_SOMATIC_STATUS] != "Confirmed somatic variant": continue

It then creates .csv files for each sample group in a new directory and outputs the following information into the following format:

| Position    | Sample Name | Strand:Substitution | Count |
|:-----------:|:-----------:|:-------------------:|:-----:|
| 12:71994489 | 1_RESISTANT | +:A>G               |   2   |
| ...         | ...         | ...                 | ...   |

Notes:
- Position indicates the genomic position, in the Chromosome number:Base position format.
- A '+' strand indicates that the mutation position is located on the forward strand.
- A '-' strand indicates it is located on the other strand, thus the complementary base must be searched (In this case: T) and the reverse complement sequence must be written to output (ex: ATA where T is the lesion site).
- Substitution is formatted as original base>mutation.
- The count of 2 in the example represents that a A>G mutation occurred twice at 12:71994489 for the 1_RESISTANT sample group.
- Any tandem base substitutions that are read are split up into individual entries occurring at their respective positions.

Execute the script using the following line as an example: `python COSMIC_csv_reader.py -i V85_38_MUTANT.csv -o ./samples/`


`COSMIC_sample_printer.py`<br>
A small portion of COSMIC_csv_reader.py that filters by certain parameters and prints the samples to console. Used primarily for searching for details of particular samples.<br>
<br>
Execute the script using the following line as an example: `python COSMIC_sample_printer.py -i V85_38_MUTANT.csv


### Finding sequence contexts within the human genome

`context_finder.py`<br>
Reads the .csv files outputted by COSMIC_csv_reader.py and uses pyfaidx to locate the sequence context of the mutation position. The program calculates and prints the percent of successful matches of the original base to genomic position. As the program counts the number of occurrences of a sequence context occurring at a certain genomic position within an individual sample, a great amount of memory is required and it is recommended that the script run with about 4 GB of memory. The sequence contexts for each sample are outputted into .csv files within a new directory. If a base does not match to the position, it is not written to the outputted to its respective .csv file. The format of each outputted file is different than those outputted by COSMIC_csv_reader.py to allow create_spectra.sh (specifically mutpos_manual.py) to read and create spectra.<br>
<br>

The format of the output .csv files should look like this:

| Position    | Context | Substitution | Count |
|:-----------:|:-------:|:------------:|:-----:|
| 12:71994489 | CAG     | A>G          | 2     |
| ...         | ...     | ...          | ...   |

Execute the script using the following line as an example: `python context_finder.py -f hg38.fa -i ./samples/ -o ./samples_contexted/`

### Copying chromosome sequences to a .fasta file for threemer count comparison

`pyfaidx_extraction.py`<br>
A script that uses pyfaidx to quickly read chromosome sequences (>chr1, >chr2, ..., >chrX, >chrY) from the .fasta file containing the human genome, hg38.fa, and output them to a new .fasta file called hg38_chromosomes.fa. This script was made to see if the outputted reference counts .txt of the threemer_count_jellyfish.sh script was impacted by .fasta entries within hg38.fa that were not relevant to the point mutation data. While there is some difference, proportionality for each threemer count was maintained.<br>
<br>
Execute the script using the following line: `python pyfaidx_extraction.py`

### Counting of threemers in the human genome

`threemer_count_jellyfish.sh`<br>
Uses jellyfish to quickly count the number of threemers within the human genome (hg38.fa or hg38_chromosomes.fa) and outputs the counts into mer_counts.txt file. The .txt file is formatted into two tab-separated columns so that it can be read by the create_spectra.sh script. Jellyfish will also output a histogram (which I admittedly do not know how to open). The script was run for both hg38.fa and hg38_chromosomes.fa. Although the sequence counts were all higher for hg38.fa, the difference between counts for both were proportional.<br> 
<br>
Run this script on ROUS using the following line as an example: `qsub threemer_count_jellyfish hg38.fa`

### Creation of Spectra for Each Sample

`create_spectra.sh`<br>
Counts the number of .csv entries (unique mutations) within each contexted sample .csv file and only creates spectra for sample .csv files that contain 50 or more entries. The header of the .csv file does not count as an entry. The spectra are specifically created by the lab's in-house script, mutpos_manual.py. For each sample group with >= 50 unique mutations, the script will output a mutation spectrum and .csv file into a new directory.

Execute the script using the following line as an example: `./create_spectra.sh ./samples_contexted ../ref/hg38_chromosomes_ref_counts.txt ./sample_contexted_figures`<br>
Where the format is ./create_spectra.sh {input contexted .csv dir} {reference count .txt made from threemer_count_jellyfish.sh} {output spectra and .csv files}

### Creation of UHC Dendrograms and Heatmaps for Visual Comparison

`create_uhc_dendrogram_heatmap.py`<br>
Performs unsupervised hierarchial clustering upon .csv sample files placed in a directory. The script uses functions from the in-house script, cospec.py, to create a dendrogram and heatmap of cosine similarity among samples. For the heatmap, it is intended to take an input of 10 or so samples. For the dendrogram, it is recommended to change the function in cospec.py to match the following code if you are working with hundreds of samples:  

In [ ]:
def plot_uhc_dendrogram(spec_list, cluster_names):
    spectra = []
    for sig in spec_list:
        spectra.append(list(spec_list[sig].values()))
    data = spectra
    labels = cluster_names
    linkages = uhc_cluster(spec_list)
    with plt.rc_context({'lines.linewidth':0.1}): #Helpeful to make the tree more interpretable
        dendrogram(linkage(pd.DataFrame(data, [labels, ['']*len(labels)]),
                                 method='ward',
                                 metric='cosine'),
                         no_labels=False,
                         labels=labels,
                         leaf_rotation=90,
                         leaf_font_size=1, #Needs to be very small to make labels readable
                         link_color_func=lambda x:'k')
    plt.show()
    plt.savefig('file.pdf', dpi=450) #File is saved as file.pdf in the same directory containing cospec.py

Execute the script using the following line as an example: `python create_uhc_dendrogram_heatmap.py -i ../../muts_csv/`


## Author

Scripts `cospec.py` and `mutpos_manual.py` were written by klkim [at] mit.edu<br>
All other scripts featured were written by jggatter [at] mit.edu<br>
pyfaidx (https://pythonhosted.org/pyfaidx/) <br>
jellyfish (http://www.cbcb.umd.edu/software/jellyfish/)